In [34]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.filterwarnings('ignore')

In [35]:
df = pd.read_csv('gsearch_jobs.csv')

In [36]:
df = df.drop(df.columns[[1,2]], axis=1)

In [37]:
df

,Unnamed: 0,company_name,location,via,description,extensions,job_id,thumbnail,posted_at,schedule_type,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,Cisco Meraki,Anywhere,via LinkedIn,"As the leader in cloud-managed IT, Cisco Merak...","['22 hours ago', 'Work from home', 'Full-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,22 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['tableau', 'sql', 'aws']"
1,1,Midwest Staffing,Anywhere,via LinkedIn,As a Senior Business Analyst you will contribu...,"['53 minutes ago', 'Work from home', 'Full-tim...",eyJqb2JfdGl0bGUiOiJFbnRyeSBMZXZlbCAtIEJ1c2luZX...,https://encrypted-tbn0.gstatic.com/images?q=tb...,53 minutes ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,"Amyx, Inc.",United States,via Indeed,Overview:\n\nAmyx is seeking to hire a Data An...,"['3 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QvUmVzZWFyY2...,NaN,3 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['sql', 'azure', 'excel']"
3,3,Upwork,Anywhere,via Upwork,I am looking for someone to help me build an A...,"['23 hours ago', 'Work from home', 'Contractor']",eyJqb2JfdGl0bGUiOiJEYXRhIGFuYWx5c3QvQUkgZXhwZX...,NaN,23 hours ago,Contractor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['excel']
4,4,Bank Street College of Education,United States,via Higher Education Recruitment Consortium (H...,Position Vacancy – Data Analyst to support the...,"['20 hours ago', 'Full-time']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,20 hours ago,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['excel', 'powerpoint']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37175,37175,EDWARD JONES,"Houstonia, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '76,798–130,764 a year', 'Ful...",eyJqb2JfdGl0bGUiOiJNYXJrZXRpbmcgRGF0YSBcdTAwMj...,NaN,23 hours ago,Full-time,...,NaN,76798–130764,a year,103781.0,76798.0,130764.0,NaN,103781.0,103781.0,"['power_bi', 'excel', 'r', 'python', 'tableau'..."
37176,37176,EDWARD JONES,"Marshfield, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]
37177,37177,EDWARD JONES,"High Point, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]
37178,37178,EDWARD JONES,"Calhoun, MO",via My ArkLaMiss Jobs,"At Edward Jones, we help clients achieve their...","['23 hours ago', '106,916–182,047 a year', 'Fu...",eyJqb2JfdGl0bGUiOiJMZWFkLURhdGEgQW5hbHlzdCIsIm...,NaN,23 hours ago,Full-time,...,NaN,106916–182047,a year,144481.5,106916.0,182047.0,NaN,144481.5,144481.5,[]


In [38]:
df_clean = df.drop(columns = ['salary','via', 'description','extensions', 'job_id', 'thumbnail', 'posted_at', 'search_term', 'date_time', 'commute_time']).copy()

In [39]:
salary_columns = ['salary_pay', 'salary_rate', 'salary_avg', 'salary_min', 'salary_max', 'salary_hourly', 'salary_yearly']
df_clean.dropna(subset = salary_columns, how='all', inplace=True)

In [40]:
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Full-time', case = False, na=False), 'Full-time', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Part-time', case = False, na=False), 'Part-time', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].str.contains('Contractor', case=False, na=False), 'Contractor', df_clean['schedule_type'])
df_clean['schedule_type'] = np.where(df_clean['schedule_type'].isna(), 'Full-time', df_clean['schedule_type'])
df_clean['work_from_home'] = np.where(df_clean['work_from_home'].isna() , 'False', df_clean['work_from_home'])
df_clean['work_from_home'] = df_clean['work_from_home'].astype(str)

In [41]:
for k, row in df_clean.iterrows():
    if row['salary_rate'] == 'a year':
        df_clean.at[k,'yearly_min'] = row['salary_min']
        df_clean.at[k,'yearly_max'] = row['salary_max']
    ### Assuming someone works 40 hours and 52 weeks
    if row['salary_rate'] == 'an hour':
        df_clean.at[k,'yearly_min'] = row['salary_min'] * 40 * 52
        df_clean.at[k,'yearly_max'] = row['salary_max'] * 40 * 52
    ### Assuming someone works the full year
    if row['salary_rate'] == 'a month':
        df_clean.at[k,'yearly_min'] = row['salary_min'] * 12
        df_clean.at[k,'yearly_max'] = row['salary_max'] * 12

In [42]:
for k, row in df_clean.iterrows():
    if (isinstance(row['location'],str)) and row['location'].strip().lower() in ['united states', 'anywhere'] or pd.isna(row['location']) or row['work_from_home'] == 'True':
        df_clean.at[k,'locations'] = 'Remote'
    else:
        df_clean.at[k,'locations'] = row['location']

In [43]:
for index, row in df_clean.iterrows():
    if row['work_from_home'] == 'False' and row['locations'] == 'Remote':
        df_clean.at[index, 'locations'] = 'Unknown'

In [44]:
df_clean['min_yearly'] = df_clean['yearly_min']

In [45]:
df_clean['max_yearly'] = df_clean['yearly_max']

In [46]:
quantile_min = df_clean['min_yearly'].quantile(0.025)
quantile_max = df_clean['max_yearly'].quantile(0.975)

df_clean = df_clean[
    (df_clean['min_yearly'] >= quantile_min) & 
    (df_clean['min_yearly'] <= quantile_max) & 
    (df_clean['max_yearly'] >= quantile_min) & 
    (df_clean['max_yearly'] <= quantile_max)
]

df_clean.drop(['yearly_min', 'yearly_max'], axis=1, inplace=True)

In [47]:
df_clean.drop(columns = ['Unnamed: 0','location','salary_pay', 'salary_avg', 'salary_min', 'salary_max', 'salary_hourly', 'salary_yearly', 'salary_standardized',], inplace=True)


In [48]:
df_clean['state'] = df_clean['locations'].apply(lambda location: location.split(',')[-1].strip() if ',' in location else location)

In [49]:
state_code = {
    'Kansas': 'KS',
    'Missouri': 'MO',
    'Oklahoma': 'OK',
    'AR': 'AR',
    'CA': 'CA',
    'KS': 'KS',
    'NE': 'NE',
    'NJ': 'NJ',
    'TX': 'TX',
    'Remote': 'Remote',
    'Unknown': 'Unknown'}

In [57]:
for index, row in df_clean.iterrows():
    cleaned_state = row['state'].strip()
    if cleaned_state in state_code:
        df_clean.at[index, 'state'] = state_code[cleaned_state]

In [51]:
for index, row in df_clean.iterrows():
    if row['description_tokens'] == '[]':
        df_clean.at[index, 'description_tokens'] = None

In [52]:
## Creating a DataFrame with all unique skills required
df_skill = pd.DataFrame()
def safe_literal_eval(x):
    try:
        if x is not None:
            return ast.literal_eval(x)
        else:
            return []
    except (SyntaxError, ValueError):
        return []
df_clean['description_tokens'] = df_clean['description_tokens'].apply(safe_literal_eval)
unique_skills = set()
for skill_list in df_clean['description_tokens']:
    for skill in skill_list:
        unique_skills.add(skill)
for skill in unique_skills:
    counts = df_clean['description_tokens'].apply(lambda x:
                                                  x.count(skill))
    df_skill[skill] = counts
df_clean.drop(columns = ['description_tokens'], inplace=True)







In [53]:
df_skill.to_csv('Skills.csv', sep=',', index=False, encoding='utf-8')

In [54]:
df_clean.to_csv('Cleaned.csv', sep=',', index=False, encoding='utf-8')

In [58]:
df_clean['state'].unique()

array(['Remote', 'Unknown', 'MO', 'OK', 'KS', 'AR', 'NJ', 'NE', 'CA',
       'TX'], dtype=object)